In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
from keras.models import load_model
import numpy as np
import cv2
import math

MODEL_PATH = "./unet.h5"  #neural network is in the models folder: https://github.com/AIican/Automatic-Measuring-of-Weld-Nuggets/tree/main/models

def circle_mask(img,offset=0):
  height,width = img.shape[:2]
  r = math.floor(min(width,height)/2)
  if min(width,height) == height:
    offset_w = offset
    offset_h = 0
  elif min(width,height) == width:
    offset_w = 0
    offset_h = offset
  else:
    offset_w = offset
    offset_h = offset
  circle_img = np.zeros((height,width), np.uint8)
  cv2.circle(circle_img,(int(width/2), int(height/2)), r, 255,thickness=-1)
  if offset is not 0:
    cv2.circle(circle_img,(math.ceil((1+offset_w)*width/2), math.ceil((1+offset_h)*height/2)), r, 255,thickness=-1)
    cv2.circle(circle_img,(math.floor((1-offset_w)*width/2), math.floor((1-offset_h)*height/2)), r, 255,thickness=-1)
  return cv2.bitwise_and(img, img, mask=circle_img)

def crop(image,x,y,w,h):
  return image[y: y + h, x: x + w]

def preprocess(img,offset=0.02):
  height,width = img.shape[:2]
  r = math.ceil(height*(1+2*offset))
  x = int((width - r)/2)
  img = circle_mask(img,offset)
  img = crop(img,x,0,r,height)
  diff = img.shape[1]-img.shape[0]
  img = cv2.copyMakeBorder(img, diff//2, diff//2, 0,0, cv2.BORDER_CONSTANT, value=[0,0,0])
  return x,diff//2,img

#extract pixel - mm ratio
def getPixelLength(img, new_width=128,yCrop=0.85,xCrop=3, mm=2):
  h,w = img.shape[:2]
  y = int(round(yCrop*h))
  cropped = crop(img,0,y,w//xCrop,h-y)
  scale_gray = cv2.cvtColor(cropped, cv2.COLOR_BGR2GRAY)
# adjust the second value of the next line to tune the detection
  ret, thresh = cv2.threshold(scale_gray, 210, 255, cv2.THRESH_BINARY)
  contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
# filter noisy detection
  contours = [c for c in contours if cv2.contourArea(c) > 100]
# sort from by (y, x)
  contours.sort(key=lambda c: (cv2.boundingRect(c)[1], cv2.boundingRect(c)[0]))
# work on the segment
  #cv2.rectangle(cropped, cv2.boundingRect(contours[-1]), (0,255,0), 2)
  x_,y_,w_,h_ = cv2.boundingRect(contours[-1])
  return w/(new_width*w_/mm)


In [ ]:
image = gr.inputs.Image()
model = load_model(MODEL_PATH)
def autoWSP(image):  
  original_height = image.shape[1]
  _x,_diff2,img = preprocess(image)
  size = img.shape
  img = cv2.resize(img, (128, 128), interpolation = cv2.INTER_AREA) 
  inp = img.reshape((-1, 128, 128, 3))
  preds_prob = model.predict(inp, verbose=1)
  preds = (preds_prob > 0.95).astype(np.uint8)
  try:
    k = getPixelLength(image)
  except:
    k = 1/12
  A = preds.sum()*(k**2)  #preds_prob.sum()i da dene
  D = 2*math.sqrt(A/np.pi)
  mask = np.squeeze(preds) * 255
  mask = cv2.resize(mask, (size[0],size[1]), interpolation = cv2.INTER_AREA) 
  mask = crop(mask,0,_diff2,original_height,mask.shape[0]-_diff2*2)
  mask = cv2.copyMakeBorder(mask, 0, 0, _x,_x, cv2.BORDER_CONSTANT, value=[0,0,0])
  edged = cv2.Canny(mask, 30, 200) 
  
# Finding Contours 
# Use a copy of the image e.g. edged.copy() 
# since findContours alters the image 
  contours, hierarchy = cv2.findContours(edged,  
    cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE) 
# Draw all contours 
# -1 signifies drawing all contours 
  cv2.drawContours(image, contours, -1, (0, 255, 0), 3) 
  return image, "Schweißpunktfläche beträgt " + str(round(A, 2)) + " mm2", "Schweißpunktdurchmesser beträgt " + str(round(D, 2)) + " mm"

iface = gr.Interface(autoWSP, image, outputs=[gr.outputs.Image(label="Prediction"),"text","text"])
iface.launch(debug=False)